#  DQN介绍-Q learnin的现实增强版
前面我们介绍了强化学习中的 q-learning，我们知道对于 q-learning，我们需要使用一个 Q 表来存储我们的状态和动作，每次我们使用 agent 不断探索环境来更新 Q 表，最后我们能够根据 Q 表中的状态和动作来选择最优的策略。但是使用这种方式有一个很大的局限性，如果在现实生活中，情况就会变得非常的复杂，我们可能有**成千上万个 state，甚至是无穷无尽有可能是无穷无尽的 state**，对于这种情况，我们不可能将所有的 state 都用 Q 表来存储，那么我们该如何解决这个问题呢？

一个非常简单的办法就是使用深度学习来解决这个问题，所以出现了一种新的网络，叫做 Deep Q Networks，将 Q learning 和 神经网络结合在了一起，对于每一个 state，我们都可以使用神经网络来计算对应动作的值，就不在需要建立一张表格，而且网络更新比表格更新更有效率，获取结果也更加高效。
![](https://camo.githubusercontent.com/ad6e45cbf6cacf0ec753f4e4f03e9d352f701b33/68747470733a2f2f7773342e73696e61696d672e636e2f6c617267652f303036744b665463677931666e6936366174366a626a3330786f3067316a75742e6a7067)

## 一. DQN简单实现
下面我们使用 open ai gym 环境中的 CartPole 来尝试实现一个简单的 DQN。

**游戏规则**:

cart pole即车杆游戏，游戏如下，很简单，游戏里面有一个小车，上有竖着一根杆子。小车需要左右移动来保持杆子竖直。如果杆子倾斜的角度大于15°，那么游戏结束。小车也不能移动出一个范围（中间到两边各2.4个单位长度）。

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import gym

In [2]:
#定义一些超参数
batch_size = 32
lr = 0.01
epsilon = 0.9
gamma = 0.9
target_replace_iter = 100
memory_capacity = 2000
env = gym.make('CartPole-v0')
env = env.unwrapped
n_actions = env.action_space.n
n_states = env.observation_space.shape[0]

In [3]:
print('number of actions are: {}'.format(n_actions))
print('number of states are: {}'.format(n_states))

number of actions are: 2
number of states are: 4


这里使用 gym 自带的环境，关于 CartPole 的一些参数，我们同样可以像上一节课 MountainCar 一样找到，地址 在这里，通过这里可以看到，这个环境的观测的 4 个值分别是板的位置，板的速度，杆的角度以及杆顶端的速度，我们能够采取的动作就是 2 个，分别是向左移动杆和向右移动杆。

本质上，我们其实可以不用关心状态到底是什么，这些状态都可以作为神经网络的输入，输出就是每个动作的 value，可以让神经网络自己学会建立一个类似 Q 表的东西。

In [4]:
class q_net(nn.Module):
    def __init__(self,hidden=50):
        super(q_net,self).__init__()
        self.fc=nn.Sequential(nn.Linear(n_states,hidden),
                             nn.ReLU(True),
                             nn.Linear(hidden,n_actions))
        
        #指定初始化方式
        nn.init.normal(self.fc[0].weight,std=0.1)# 使用标准差是 0.1 的正态分布初始化
        nn.init.normal(slef.fc[2].weight,std=0.1)# 使用标准差是0.1的正态分布初始化
    def forward(self,x):
        actions_value=self.fc(x)
        return actions_value

接下来我们定义一个 DQN 的过程作为一个类，在这个类中，我们会定义 dqn 的学习过程，dqn 的更新过程和之前讲个 q learning 的过程是很相同的

![](https://camo.githubusercontent.com/252c47e9d29a3a9d838d0241cd4595fa6f9c142e/68747470733a2f2f7773312e73696e61696d672e636e2f6c617267652f303036744e6337396c7931666e686c3638653973706a3330656b3062697766772e6a7067)